Import library yang diperlukan

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

Membaca file CSV dan mengonversinya menjadi objek DataFrame (df)

In [2]:
df = pd.read_csv('food_ingredients_and_allergens.csv')

Eksplorasi data

In [3]:
# Menampilkan ringkasan singkat tentang struktur dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Food Product     399 non-null    object
 1   Main Ingredient  399 non-null    object
 2   Sweetener        399 non-null    object
 3   Fat/Oil          399 non-null    object
 4   Seasoning        399 non-null    object
 5   Allergens        399 non-null    object
 6   Prediction       398 non-null    object
dtypes: object(7)
memory usage: 21.9+ KB


In [4]:
# Menampilkan beberapa baris pertama dari dataset
df.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
1,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
2,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
3,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
4,Cheddar Cheese,Cheese,None,None,Salt,Dairy,Contains


In [5]:
# Menampilkan ukuran dataset (jumlah_baris, jumlah_kolom)
df.shape

(399, 7)

In [6]:
# Menghitung jumlah baris yang memiliki duplikat (data yang sama)
sum(df.duplicated())

90

In [7]:
# Menghapus baris yang merupakan duplikat
df.drop_duplicates(inplace = True)
df.shape

(309, 7)

In [8]:
# Menghitung jumlah nilai null atau missing dalam setiap kolom dari dataset
df.isnull().sum()

Food Product       0
Main Ingredient    0
Sweetener          0
Fat/Oil            0
Seasoning          0
Allergens          0
Prediction         1
dtype: int64

In [9]:
# Menghapus baris yang mengandung nilai null atau missing dalam dataset
df.dropna(inplace=True)
df.isnull().sum()

Food Product       0
Main Ingredient    0
Sweetener          0
Fat/Oil            0
Seasoning          0
Allergens          0
Prediction         0
dtype: int64

Implementasi sistem rekomendasi makanan berdasarkan informasi alergen yang dimiliki oleh setiap pengguna (user).
1. Mendefinisikan alergen untuk masing-masing pengguna.
2. Menggabungkan dua kolom dalam dataset, yaitu kolom 'Main Ingredient' dan 'Allergens' menjadi satu kolom baru dengan nama 'Ingredients_Allergens'.
3.  Mengubah teks bahan-bahan dan informasi alergen yang terdapat dalam kolom 'Ingredients_Allergens' menjadi representasi numerik dengan skema TF-IDF (Term Frequency-Inverse Document Frequency). Dengan menggunakan representasi numerik ini, kita dapat mengukur kemiripan antara bahan-bahan yang dimiliki oleh pengguna dengan bahan-bahan pada setiap makanan dalam dataset.
4.  Menghitung kemiripan kosinus (cosine similarity) antara setiap pasang makanan dalam dataset berdasarkan representasi numerik TF-IDF yang telah dihasilkan sebelumnya.
5.  Membuat dataframe yang berisi hasil kemiripan antar makanan satu dengan makanan lain.
6. Mendefinisikan fungsi untuk merekomendasikan makanan.

In [10]:
# Langkah 1
user_allergens = {
    'User1': 'Almonds, Wheat, Dairy',
    'User2': 'Chicken, Wheat, Celery',
    'User3': 'Dairy'
}

# Langkah 2
df['Ingredients_Allergens'] = df['Main Ingredient'] + ', ' + df['Allergens']

# Langkah 3
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Ingredients_Allergens'])

# Langkah 4
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Langkah 5
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['Food Product'], columns=df['Food Product'])

# Langkah 6
def recommend_food(user_allergen, n=5):
    # Menggabungkan bahan-bahan dan informasi allergens pengguna dalam satu teks
    user_teks = user_allergen + ', ' + user_allergens[user_allergen]

    # Mengubah teks pengguna menjadi vektor TF-IDF
    user_vector = tfidf_vectorizer.transform([user_teks])

    # Menghitung similarity antara makanan dengan preferensi allergi pengguna
    similarity_scores = linear_kernel(user_vector, tfidf_matrix).flatten()

    # Mengurutkan makanan berdasarkan similarity dengan preferensi allergi pengguna
    related_food_indices = similarity_scores.argsort()[::-1]

    # Mengambil n makanan teratas sebagai rekomendasi
    recommended_food = df['Food Product'].iloc[related_food_indices][:n].tolist()

    return recommended_food

def main():
    # Menampilkan semua rekomendasi makanan untuk setiap pengguna
    all_user_recommendations = {}
    for user in user_allergens:
        recommended_food = recommend_food(user)
        all_user_recommendations[user] = recommended_food

    print(f"\nRekomendasi makanan untuk setiap pengguna :")
    for user, recommended_food_list in all_user_recommendations.items():
        print(f"{user}: {recommended_food_list}")

if __name__ == "__main__":
    main()


Rekomendasi makanan untuk setiap pengguna :
User1: ['Almond Cookies', 'Chicken Parmesan', 'Chicken Quesadilla', 'Chicken Enchiladas', 'Chicken Caesar Salad']
User2: ['Chicken Noodle Soup', 'Chicken Pot Pie', 'Chicken Noodle Casserole', 'Chicken Parmesan', 'Chicken Quesadilla']
User3: ['Chicken Fettuccine Alfredo', 'Chicken Tikka Masala', 'Spinach and Feta Stuffed Chicken', 'Butter Chicken', 'Chicken Malai Tikka']


Evaluasi menggunakan metode Hit Rate, menghitung berapa banyak pengguna yang menerima rekomendasi makanan yang aman (tidak mengandung alergen) dan membaginya dengan jumlah total pengguna.


In [11]:
# Mendefinisikan fungsi hit_rate
def hit_rate(all_user_recommendations, user_allergens):
    num_safe_recommendations = 0

    for user, recommended_food_list in all_user_recommendations.items():
        if set(recommended_food_list).isdisjoint(user_allergens[user]):
            num_safe_recommendations += 1

    total_users = len(all_user_recommendations)

    hit_rate_score = num_safe_recommendations / total_users

    return hit_rate_score

# Data user_allergens dan all_user_recommendations
user_allergens = {
    'User1': {'Almonds' , 'Wheat', 'Dairy'},
    'User2': {'Chicken' , 'Wheat', 'Celery'},
    'User3': {'Dairy'},
}

all_user_recommendations = {
    'User1': ['Almond Cookies', 'Almond Cookies', 'Chicken Caesar Salad', 'Chicken Enchiladas', 'Chicken Enchiladas'],
    'User2': ['Chicken Noodle Soup', 'Chicken Noodle Soup', 'Chicken Pot Pie', 'Chicken Noodle Casserole', 'Chicken Enchiladas'],
    'User3': ['Chicken Piccata', 'Chicken Korma', 'Chicken Fettuccine Alfredo', 'Chicken Caesar Wrap', 'Tandoori Chicken'],
}

# Memanggil fungsi hit_rate dengan data all_user_recommendations dan user_allergens
hit_rate_score = hit_rate(all_user_recommendations, user_allergens)

# Menampilkan hasil hit rate
print("Hit Rate:", hit_rate_score)

Hit Rate: 1.0
